#### include library code

In [ ]:
using Random

#### define agents

In [ ]:
mutable struct ComplexHuman
    # var :: type
    migrant :: Bool
    employed :: Bool
    industry :: Int
    origin :: Country
    residence :: Country
    contacts :: Vector{ComplexHuman}
end

#### define countries

In [ ]:
mutable struct Country
    # this rate w/ be used in migration decision
    migration_rate :: Float64
    job_market :: Int
end

#### define industries and job market

In [ ]:
mutable struct Industry
    num_jobs :: num_jobs
end


function update_job_market!()
    for i in length(pop):
        if pop[i].employed == false
            return
        end
        if pop[i].employed == true
            Industry.num_jobs += 1
        end
    end
end

    # needs to be like hey no more jobs
    # you can't get these jobs they're gone

#### define industries and job market